In [3]:
import mrcfile
import starfile
import os
import numpy as np
mask_path = "20s_10025/MaskCreate/job029/mask.mrc"
meta_data_path = '20s_10025/Refine3D/job026/run_data.star'
output_data_dir = 'train_data/20s/mask_mrcs/'

meta_data = starfile.read(meta_data_path)
meta_data


,rlnVoltage,rlnDefocusU,rlnDefocusV,rlnDefocusAngle,rlnSphericalAberration,rlnDetectorPixelSize,rlnCtfFigureOfMerit,rlnMagnification,rlnAmplitudeContrast,rlnImageName,...,rlnLogLikeliContribution,rlnRandomSubset,rlnParticleName,rlnOriginalParticleName,rlnNrOfSignificantSamples,rlnNrOfFrames,rlnMaxValueProbDistribution,rlnCtfMaxResolution,rlnBeamTiltX,rlnBeamTiltY
0,300.0,12479.890767,12581.598853,-5.579550,2.7,0.66,-0.11592,10000.0,0.1,000001@Extract/job011/mrc/14sep05c_00024sq_000...,...,669428.1,1,1,000001@Particles/Micrographs/14sep05c_00024sq_...,13,38,0.146965,2.722,0.144042,0.107795
1,300.0,12059.890767,12161.598853,-5.579550,2.7,0.66,-0.11592,10000.0,0.1,000002@Extract/job011/mrc/14sep05c_00024sq_000...,...,669609.4,1,2,000002@Particles/Micrographs/14sep05c_00024sq_...,25,38,0.072918,2.722,0.144042,0.107795
2,300.0,12479.890767,12581.598853,-5.579550,2.7,0.66,-0.11592,10000.0,0.1,000003@Extract/job011/mrc/14sep05c_00024sq_000...,...,669564.8,1,3,000003@Particles/Micrographs/14sep05c_00024sq_...,4,38,0.301979,2.722,0.144042,0.107795
3,300.0,12523.890767,12625.598853,-5.579550,2.7,0.66,-0.11592,10000.0,0.1,000004@Extract/job011/mrc/14sep05c_00024sq_000...,...,669533.0,2,7,000007@Particles/Micrographs/14sep05c_00024sq_...,7,38,0.224601,2.722,0.144042,0.107795
4,300.0,12403.890767,12505.598853,-5.579550,2.7,0.66,-0.11592,10000.0,0.1,000005@Extract/job011/mrc/14sep05c_00024sq_000...,...,669467.0,2,11,000011@Particles/Micrographs/14sep05c_00024sq_...,8,38,0.413934,2.722,0.144042,0.107795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49949,300.0,17381.785776,17564.526780,38.063196,2.7,0.66,-0.16402,10000.0,0.1,000230@Extract/job011/mrc/14sep05c_c_00007gr_0...,...,669915.8,2,2262869,000311@Particles/Micrographs/14sep05c_c_00007g...,9,38,0.603843,3.589,0.144042,0.107795
49950,300.0,17777.785776,17960.526780,38.063196,2.7,0.66,-0.16402,10000.0,0.1,000231@Extract/job011/mrc/14sep05c_c_00007gr_0...,...,669603.5,1,2262872,000314@Particles/Micrographs/14sep05c_c_00007g...,17,38,0.303726,3.589,0.144042,0.107795
49951,300.0,17593.785776,17776.526780,38.063196,2.7,0.66,-0.16402,10000.0,0.1,000232@Extract/job011/mrc/14sep05c_c_00007gr_0...,...,669600.1,2,2262875,000317@Particles/Micrographs/14sep05c_c_00007g...,14,38,0.170574,3.589,0.144042,0.107795
49952,300.0,17541.785776,17724.526780,38.063196,2.7,0.66,-0.16402,10000.0,0.1,000233@Extract/job011/mrc/14sep05c_c_00007gr_0...,...,669529.7,1,2262877,000319@Particles/Micrographs/14sep05c_c_00007g...,13,38,0.119440,3.589,0.144042,0.107795


In [14]:
grouped = list(meta_data.groupby('rlnMicrographName'))
for name, group in grouped:
    mrc_slices = []
    for index, row in group[:3].iterrows():
        particle_idx = row['rlnImageName'].split('@')[0]
        mrc_name = row['rlnImageName'].split('@')[1].split('/')[-1][:-1]
        single_slice_name = f'{particle_idx}--{mrc_name}'
        rot = row['rlnAngleRot']
        tilt = row['rlnAngleTilt']
        psi = row['rlnAnglePsi']
        xoff = row['rlnOriginX']
        yoff = row['rlnOriginY']
        os.system(f'relion_project \
            --i {mask_path} --o {output_data_dir}{single_slice_name}\
            --rot {rot} --tilt {tilt}\
            --psi {psi} --xoff {xoff}\
            --yoff {yoff}')
        with mrcfile.open(f'{output_data_dir}{single_slice_name}') as mrc:
            mrc_slices.append(mrc.data)
        os.system(f'rm {output_data_dir}{single_slice_name}')
    with mrcfile.open(f"{output_data_dir}{name.split('/')[-1]}s", mode='w+') as mrc_stack:
        mrc_stack.set_data(np.array(mrc_slices))
    break

 Reading map: 20s_10025/MaskCreate/job029/mask.mrc
 Done reading map!


         The value in the input image header (= 0.66) is used instead.


 Done writing train_data/20s/mask_mrcs/000001--14sep05c_00024sq_00003hl_00002es_c.mrc
 Reading map: 20s_10025/MaskCreate/job029/mask.mrc
 Done reading map!


         The value in the input image header (= 0.66) is used instead.


 Done writing train_data/20s/mask_mrcs/000002--14sep05c_00024sq_00003hl_00002es_c.mrc
 Reading map: 20s_10025/MaskCreate/job029/mask.mrc
 Done reading map!


         The value in the input image header (= 0.66) is used instead.


 Done writing train_data/20s/mask_mrcs/000003--14sep05c_00024sq_00003hl_00002es_c.mrc


ValueError: File 'train_data/20s/mask_mrcs/14sep05c_00024sq_00003hl_00002es_c.mrcs' already exists; set overwrite=True to overwrite it

In [17]:
grouped = list(meta_data.groupby('rlnMicrographName'))
grouped[0][0]


'mrc/14sep05c_00024sq_00003hl_00002es_c.mrc'